In [1]:
import pandas as pd
from pathlib import Path

In [2]:
eval_path = "../eval"

In [3]:
depth_dirs = ["depth", "multi_tasks"]
dfs = []
for dir in depth_dirs:
    for file in Path(eval_path).joinpath(dir).glob("*.json"):
        df = pd.read_json(file)

        dfs.append(df)

depth_df = pd.concat(dfs, ignore_index=True)
metadata_df = pd.json_normalize(depth_df["metadata"])
depth_df = pd.concat([depth_df.drop(columns=["metadata"]), metadata_df], axis=1)

depth_df["model"] = depth_df["model_name"].apply(lambda x: x.split("_")[0])
depth_df["augmentation"] = depth_df.apply(
    lambda row: row["augmentation"]
    if pd.notna(row["augmentation"])
    else "augmented" in row["model"],
    axis=1,
)
depth_df["rmse"] = depth_df.apply(
    lambda row: row["metric_value"] if row["metric_name"] == "rmse" else row["rmse"],
    axis=1,
)
depth_df_cols = [
    "model",
    "rmse",
    "augmentation",
    "learning_rate",
] + depth_df.columns.drop(
    ["model", "rmse", "metric_name", "metric_value", "augmentation", "learning_rate"]
).tolist()
depth_df = depth_df[depth_df_cols]

depth_df = depth_df.sort_values(by=["model"]).reset_index(drop=True)

depth_df.head(8)

,model,rmse,augmentation,learning_rate,model_name,N_EPOCHS,EARLY_STOPPING_PATIENCE,multi_label,multi_tasks_regression_loss_weight,precision,recall,f1,accuracy_p,accuracy_r,precision_p,recall_p,f1_p,precision_r,recall_r,f1_r
0,alexnet,0.080652,False,0.0001,alexnet_multi_tasks,128,16,False,NaN,0.979154,0.976944,0.976686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,alexnet,0.110612,True,0.0001,alexnet_multi_tasks_augmented_multi_label,128,16,True,NaN,NaN,NaN,NaN,0.9825,0.9975,0.973232,0.965862,0.964908,0.995392,0.996516,0.995894
2,alexnet,0.070930,False,0.0010,alexnet_depth,32,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,alexnet,0.039216,False,0.0001,alexnet_depth,32,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,alexnet,0.051154,True,0.0001,alexnet_depth_augmented,50,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,alexnet,0.107530,False,0.0001,alexnet_multi_tasks_multi_label,128,16,True,NaN,NaN,NaN,NaN,0.9775,0.9875,0.968704,0.973775,0.969215,0.983823,0.978406,0.980965
6,alexnet,0.052069,True,0.0001,alexnet_multi_tasks_augmented,128,16,False,4.0,0.985417,0.983750,0.982756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,alexnet,0.075205,True,0.0001,alexnet_multi_tasks_augmented,128,16,False,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
depth_df.to_csv("../eval/depth_model_comparison.csv", index=False)

In [5]:
pose_dirs = ["multi_tasks", "pose_single", "pose_multi"]

dfs = []
for dir in pose_dirs:
    for file in Path(eval_path).joinpath(dir).glob("*.json"):
        df = pd.read_json(file)

        dfs.append(df)

pose_df = pd.concat(dfs, ignore_index=True)
metadata_df = pd.json_normalize(pose_df["metadata"])
pose_df = pd.concat([pose_df.drop(columns=["metadata"]), metadata_df], axis=1)
pose_df["model"] = pose_df["model_name"].apply(lambda x: x.split("_")[0])
pose_df["multi_label"] = pose_df["model_name"].apply(
    lambda x: "pose_multi" in x or "multi_tasks" in x
)

depth_df["augmentation"] = depth_df.apply(
    lambda row: row["augmentation"]
    if pd.notna(row["augmentation"])
    else "augmented" in row["model"],
    axis=1,
)
pose_df["accuracy"] = pose_df.apply(
    lambda row: row["accuracy"]
    if pd.isna(row["metric_value"])
    else row["metric_value"],
    axis=1,
)


pose_df_cols = [
    "model",
    "accuracy",
    "augmentation",
    "learning_rate",
    "multi_label",
] + pose_df.columns.drop(
    [
        "model",
        "accuracy",
        "metric_value",
        "augmentation",
        "learning_rate",
        "multi_label",
    ]
).tolist()


pose_df = pose_df[pose_df_cols]
pose_df = pose_df.sort_values(by=["model"]).reset_index(drop=True)
pose_df.head(8)


,model,accuracy,augmentation,learning_rate,multi_label,model_name,metric_name,N_EPOCHS,EARLY_STOPPING_PATIENCE,multi_tasks_regression_loss_weight,...,recall,f1,accuracy_p,accuracy_r,precision_p,recall_p,f1_p,precision_r,recall_r,f1_r
0,alexnet,0.9700,False,0.0001,True,alexnet_multi_tasks,accuracy,128,16,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,alexnet,0.9500,True,0.0001,True,alexnet_pose_multi_augmented,accuracy,50,8,NaN,...,NaN,NaN,0.9575,0.9925,NaN,NaN,NaN,NaN,NaN,NaN
2,alexnet,0.9775,True,0.0001,True,alexnet_pose_multi_augmented,accuracy,95,8,NaN,...,NaN,NaN,0.9875,0.9900,0.973120,0.976984,0.974412,0.984694,0.982143,0.983179
3,alexnet,0.9800,True,0.0001,False,alexnet_pose_model_single_label_augmented,NaN,50,8,NaN,...,0.980000,0.979918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,alexnet,0.9725,False,0.0001,False,alexnet_pose_model_single_label,NaN,50,8,NaN,...,0.972500,0.971977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,alexnet,0.0250,False,0.0010,False,alexnet_pose_model_single_label,NaN,50,8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,alexnet,0.9775,False,0.0001,True,alexnet_multi_tasks,accuracy,128,16,NaN,...,0.976944,0.976686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,alexnet,0.9825,True,0.0001,True,alexnet_multi_tasks_augmented_multi_label,accuracy,128,16,NaN,...,NaN,NaN,0.9825,0.9975,0.973232,0.965862,0.964908,0.995392,0.996516,0.995894


In [6]:
pose_df.to_csv("../eval/pose_model_comparison.csv", index=False)